#Pipeline

## Table of Contents

1. `Pipeline` class
1. `fit` and `transform` methods in pipeline

### 1. `Pipeline` class

`Pipeline` can be used to chain multiple transformers (and an optional final estimator) into one. This is useful as there is often a fixed sequence of steps in processing the data.

Load libraries.

In [0]:
import sklearn as  sk
from sklearn.pipeline      import Pipeline
from sklearn.impute        import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors     import KNeighborsClassifier

Build a `Pipeline` and store it in an object `pipe`. The `Pipeline` is built using a list of _(key, value)_ pairs, where the _key_ is a string containing the name you want to give this step and _value_ is an estimator/transformer object:

In [4]:
pipe = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                 ('scaler' , MinMaxScaler()),
                 ('reduce_dim', PCA()),
                 ('knn'    , KNeighborsClassifier())
                 ])
pipe

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

The output above shows that the four transformers are chained in the order we defined in the `Pipeline`. In addition, the default init parameters are displayed.

Note: All estimators in a pipeline, except the last one, must be transformers (i.e. must have a `transform` method). The last estimator may be any type (transformer, classifier, etc.).

The transformers of a pipeline are stored as a list in the `steps` attribute:

In [5]:
pipe.steps[2]

('reduce_dim',
 PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
     svd_solver='auto', tol=0.0, whiten=False))

The final estimator is also stored in the `steps` attribute.

In [6]:
pipe.steps[-1]

('knn',
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                      metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                      weights='uniform'))

The transformers of a pipeline are also stored as a dict in `named_steps` attribute:

In [7]:
pipe.named_steps['reduce_dim']

PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

Parameters of the transformers in the pipeline can be accessed using the `set_params` method with the `<step name>__<parameter>` syntax:

In [8]:
 pipe.set_params(imputer__strategy='most_frequent') 

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

This session introduces the way to build a pipeline and to obtain parameters of the transformers in the pipeline.

###2. `fit` and `transform` methods in pipeline

In scikit-learn, estimators have a `fit` method, which learns model parameters from a training set, and a `transform` method which applies this transformation model to unseen data. `fit_transform` may be more convenient and efficient for modelling and transforming the training data simultaneously. Calling `fit` on the pipeline is the same as calling `fit` on each estimator in turn, `transform` the input and pass it on to the next step. 
 - [Pipeline process](https://ibb.co/m4sz3o)

Load libraries.

In [0]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

Prepare a sample dataset from the sklearn library, and store it in the `digits` object.

In [0]:
digits = load_digits()

Split the predictors (`digit.data`) and labels (`digits.target`) into train and test using the `train_test_split` function.

In [0]:
X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=.2, random_state=0)

Fit the pipeline using `X_train` as training data and `y_train` as target values, and store the computed parameters to the object `Pipe`.

In [12]:
pipe.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)

The above pipeline process consists of several intermediate steps (`Imputer`, `MinMaxScaler`, `PCA`) and a final estimator (`KNeighborsClassifier`). When the `fit` method of the pipeline is called with the training data:
1. The `Imputer` object is fit to the training data
1. The `Imputer` object transforms the training data
1. The `MinMaxScaler` object is fit to the transformed training data from the `Imputer` object (previous step)
1. The `MinMaxScaler` object transforms the transformed training data from the `Imputer` object
1. The `PCA` object fits the transformed data from the `MinMaxScaler` object
1. The `PCA` object transforms the transformed data from the `MinMaxScaler` object
1. The `KNeighborsClassifier` object fits the transformed data from the `PCA` object

That's all it does. In particular, the `transform` method of the `KNeighborsClassifier` __is not called__.

Apply the pipeline transformations to the test data, and score with the final estimator.

In [13]:
print('Test accuracy: %.3f' % pipe.score(X_test, y_test))

Test accuracy: 0.981


The pipeline has all the methods that the last estimator in the pipeline has, i.e. if the last estimator is a classifier, the pipeline can be used as a classifier with all the methods that classifier has. If the last estimator is a transformer, again, so is the pipeline.

Below is the `Pipeline` command that originally created the object `pipe`.

In [15]:
pipe = Pipeline([('imputer', SimpleImputer(strategy='mean')),
                 ('scaler' , MinMaxScaler()),
                 ('reduce_dim', PCA()),
                 ('knn'    , KNeighborsClassifier())
                 ])
pipe

Pipeline(memory=None,
         steps=[('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('reduce_dim',
                 PCA(copy=True, iterated_power='auto', n_components=None,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('knn',
                 KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                      metric='minkowski', metric_params=None,
                                      n_jobs=None, n_neighbors=5, p=2,
                                      weights='uniform'))],
         verbose=False)